<a href="https://colab.research.google.com/github/inamansari21/datascience/blob/main/MLR_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from numpy.polynomial.polynomial import polyfit
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.stats.tests.test_influence
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math


In [ ]:
df = pd.read_csv("50_Startups.csv")


In [ ]:
df1 = df.copy()
df1

In [ ]:
df1.describe


In [ ]:
sns.pairplot(df1)


# **correlation matrix**

In [ ]:
corrMatrix = df1.corr()


In [ ]:
sns.heatmap(corrMatrix, annot=True)


# **regression model**

In [ ]:
df1 = pd.get_dummies(df1, columns=['State'])


In [ ]:
X = df1[['R&D Spend','Administration', 'Marketing Spend', 'State_California', 'State_Florida', 'State_New York']]
Y = df1[['Profit']]


In [ ]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)


In [ ]:
model.summary()

R sq and p Value of the Model is Good and the model can be accepted. However as you can see not all variables have acceptable p value. Thus we have Multicollinearity issue in our Data Frame


# **solving multicollinearity**

# **finding cook's distance**



In [ ]:
infl = model.get_influence()


In [ ]:
summ_df = infl.summary_frame()


In [ ]:
summ_df.sort_values('cooks_d', ascending=False)


In [ ]:
infl.plot_influence()


Index 48 and 49 has highest Cook's Distance.
Thus it will be wise to drop this row from the data set



# **finding VIF**

In [ ]:
vif = pd.DataFrame()


In [ ]:
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [ ]:
vif["features"] = X.columns


In [ ]:
vif.round(1)


**As expected, Administration has the lowest variance inflation factor We would need to discard this variables to improve model and try to solve multicolinearity.
**

In [ ]:
# Removed administration fron the dataframe and Test the model
new_X = df1[['R&D Spend', 'Marketing Spend', 'State_California', 'State_Florida', 'State_New York']]


In [ ]:
new_model = sm.OLS(Y, new_X).fit()
new_predictions = new_model.predict(new_X)


In [ ]:
new_model.summary()


**once we remove "Administration" from input variables list and run the model again, all the variables are significant.
**

# **FINAL MODEL**

In [ ]:
# Removed Index with highest Cook's distance to remove the hightest influencial ROW
new_df = df1.drop(df1.index[[49,48]])


In [ ]:
new_X = new_df[['R&D Spend', 'Marketing Spend', 'State_California', 'State_Florida', 'State_New York']]
new_Y = new_df[['Profit']]


In [ ]:
final_model = sm.OLS(new_Y, new_X).fit()
predictions = final_model.predict(new_X)


In [ ]:
final_model.summary()


# **SQUARE ROOT TRANSFORMATION OF X**

In [ ]:
X_sqrt = np.sqrt(new_df[['R&D Spend', 'Marketing Spend', 'State_California', 'State_Florida', 'State_New York']])


In [ ]:
model3 = sm.OLS(new_Y, X_sqrt).fit()
predictions3 = model3.predict(X_sqrt)


In [ ]:
model3.summary()

# **square root transformation of Y**

In [ ]:
Y_sqrt = np.sqrt(new_df['Profit'])


In [ ]:
model4 = sm.OLS(Y_sqrt, new_X).fit()
predictions4 = model4.predict(new_X)


In [ ]:
model4.summary()

# **square root transformation of X and Y**

In [ ]:
model5 = sm.OLS(Y_sqrt, X_sqrt).fit()
predictions5 = model5.predict(X_sqrt)


In [ ]:
model5.summary()

# **output interpretation**
We will use Model5  as it has the best R square value 1 - p-value < 0.01
Thus the model is accepted

2 - coefficient == 1 Thus if the value of x increased by 1, the predicted value of Price will increase by 1

3 - Adj. R-sqared == 1
Thus the model explains 100% of the variance in dependent variable



In [ ]:
|